Create Amps Sqlite
===

Creates sqlite file from the amps data.

Note: no unique identifier is created for each amp in the resulting database (other than the rowid).

See original usage of the raw amps data in `sna-social-support/data_pulling/FilterAndMergeExtractedInteractions.ipynb`.

In [6]:
import numpy as np
import pandas as pd

import os
import sqlite3
from datetime import datetime

In [3]:
input_filepath = "/home/lana/shared/caringbridge/data/projects/sna-social-support/csv_data/new_amps_metadata_raw.csv"
output_filepath = "/home/lana/shared/caringbridge/data/projects/recsys-peer-match/sqlite/amps.sqlite"

In [4]:
!du -h {input_filepath}

5.0G	/home/lana/shared/caringbridge/data/projects/sna-social-support/csv_data/new_amps_metadata_raw.csv


In [12]:
amps_cols = ['user_id','site_id','journal_oid','journal_user_id','created_at','updated_at']
dtypes = {
    'user_id': int,
    'site_id': int,
    'journal_oid': str,
    'journal_user_id': float,
    'created_at': float,
    'updated_at': int
}
s = datetime.now()
amps_df = pd.read_csv(input_filepath, header=None, names=amps_cols,
                     dtype=dtypes,
                     na_values=['', 'None'])
print(datetime.now() - s)
len(amps_df)

0:00:40.143970


63314738

In [14]:
# NA values in two of the columns
amps_df.isna().sum()

user_id                 0
site_id                 0
journal_oid             0
journal_user_id    541278
created_at              2
updated_at              0
dtype: int64

In [16]:
# map the NA values to defaults
# and convert to the correct datatype
amps_df = amps_df.fillna({'journal_user_id': -1, 'created_at': 0}).astype({
    'user_id': int,
    'site_id': int,
    'journal_oid': str,
    'journal_user_id': int,
    'created_at': int,
    'updated_at': int
})
len(amps_df)

63314738

In [18]:
amps_df.dtypes

user_id             int64
site_id             int64
journal_oid        object
journal_user_id     int64
created_at          int64
updated_at          int64
dtype: object

In [19]:
def get_db(db_filename):
    db = sqlite3.connect(
            db_filename,
            detect_types=sqlite3.PARSE_DECLTYPES
        )
    db.row_factory = sqlite3.Row
    return db

conn = get_db(output_filepath)
s = datetime.now()
amps_df.to_sql('amps', conn, index=False, chunksize=1000000)
print(datetime.now() - s)

0:01:47.172553


I created the following indices:

    CREATE INDEX amps_journalOid ON amps (journal_oid);
    CREATE INDEX amps_siteId ON amps (site_id);
    CREATE INDEX amps_siteId_journalOid ON amps (site_id, journal_oid);
    CREATE INDEX amps_userId ON amps (user_id);